In [1]:
import os
import re
import pandas as pd
from sqlalchemy import create_engine
from langchain.chat_models import init_chat_model
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool, ListSQLDatabaseTool, QuerySQLCheckerTool, QuerySQLDatabaseTool
)
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain.agents.agent_types import AgentType
from langgraph.graph.state import StateGraph
from pydantic import BaseModel

In [2]:
# Step 1: Set API Key and Initialize LLM
llm = init_chat_model(model="llama3.1:8b", model_provider="ollama", temperature=0)

1. temperature controls the randomness of the model's output
2. temerature = 0 means it makes max deterministic and focused, 
3. meaning it will choose the most likely next word each time, 
4. resulting in less variation and more predictable outputs.

In [3]:
# Step 2: Fetch and Process Files from Folder
folder_path = r"D:\VSCode Programs\Final Output\Database Files"  # Change to your folder path
db_connections = {}  # Stores all database connections

In [4]:
print(f"Searching for files in: {folder_path}...\n")

for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    if file.endswith(".db"):
        print(f"Found database: {file}")
        db_name = file.replace(".db", "")
        db_connections[db_name] = SQLDatabase(create_engine(f"sqlite:///{file_path}"))
    elif file.endswith((".xlsx", ".xls", ".csv")):
        print(f"Converting {file} to SQLite...")
        db_name = os.path.splitext(file)[0].replace(" ", "_") + ".db"
        db_path = os.path.join(folder_path, db_name)
        engine = create_engine(f"sqlite:///{db_path}")
        
        if file.endswith(".csv"):
            try:
                df = pd.read_csv(file_path, encoding='utf-8')
            except UnicodeDecodeError:
                df = pd.read_csv(file_path, encoding='latin1')
            table_name = os.path.splitext(file)[0].replace(" ", "_")
            df.to_sql(table_name, con=engine, if_exists="replace", index=False)
            print(f"Created database: {db_name} with table '{table_name}'")
        else:
            xls = pd.ExcelFile(file_path)
            for sheet_name in xls.sheet_names:
                df = pd.read_excel(file_path, sheet_name=sheet_name)
                table_name = sheet_name.replace(" ", "_")
                print(f"Processing sheet: {sheet_name}")
                print(df.head())  # Print the first few rows of the DataFrame to check its content
                if not df.empty:
                    df.to_sql(table_name, con=engine, if_exists="replace", index=False)
                    print(f"Created table '{table_name}' in database: {db_name}")
                else:
                    print(f"Sheet '{sheet_name}' is empty and will not be added to the database.")
        
        db_connections[db_name.replace(".db", "")] = SQLDatabase(engine)

print("Conversion complete.")


Searching for files in: D:\VSCode Programs\Final Output\Database Files...

Converting atmspl_trip_details.csv to SQLite...
Created database: atmspl_trip_details.db with table 'atmspl_trip_details'
Found database: atmspl_trip_details.db
Converting Customer Portfolio.csv to SQLite...
Created database: Customer_Portfolio.db with table 'Customer_Portfolio'
Found database: Customer_Portfolio.db
Converting diabetes.csv to SQLite...
Created database: diabetes.db with table 'diabetes'
Found database: diabetes.db
Found database: IFDP.db
Converting landmark.csv to SQLite...
Created database: landmark.db with table 'landmark'
Found database: landmark.db
Converting Meeting Updates.csv to SQLite...
Created database: Meeting_Updates.db with table 'Meeting_Updates'
Found database: Meeting_Updates.db
Converting road.csv to SQLite...
Created database: road.db with table 'road'
Found database: road.db
Converting Stock Ageing.csv to SQLite...
Created database: Stock_Ageing.db with table 'Stock_Ageing'
Fo

In [5]:
# Step 3: Create SQL Agents for Each Database
db_agents = {}
for db_name, db in db_connections.items():
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    db_agents[db_name] = create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

1. Setting up a toolkit for interacting with an SQL database using a language model
2. Leverage the language model to generate SQL queries, interpret results, 
3. Or perform other database-related tasks more efficiently
4. list_tables_tool = ListSQLDatabaseTool(db=db): tool to list/display tables names in your SQL database
5. InfoSQLDatabaseTool(db=db): detailed information about DB, such as schema details, table structures, metadata
6. QuerySQLCheckerTool(db=db, llm=llm): validate or analyze SQL queries to ensure they are correct and optimized
7. QuerySQLDatabaseTool(db=db): facilitate running queries and retrieving results

In [6]:
# Step 4: Fetch Schema Information
schema_info = {}
for db_name, db in db_connections.items():
    print(f"\nFetching tables for {db_name}...")
    list_tables_tool = ListSQLDatabaseTool(db=db)
    tables = list_tables_tool.run("")
    schema_info[db_name] = {}
    for table in tables.split(", "):
        info_tool = InfoSQLDatabaseTool(db=db)
        schema_info[db_name][table] = info_tool.run(table)
        print(f"Schema retrieved for {table} in {db_name}")



Fetching tables for atmspl_trip_details...
Schema retrieved for atmspl_trip_details in atmspl_trip_details

Fetching tables for Customer_Portfolio...
Schema retrieved for Customer_Portfolio in Customer_Portfolio

Fetching tables for diabetes...
Schema retrieved for diabetes in diabetes

Fetching tables for IFDP...
Schema retrieved for Coal_IFDP_Daily_Index_Accuracy_Table in IFDP
Schema retrieved for Coal_IFDP_Dynamic_Pricing_MAP_Actual_Pred in IFDP
Schema retrieved for Coal_IFDP_Historic_Pricing_Monthly in IFDP

Fetching tables for landmark...
Schema retrieved for landmark in landmark

Fetching tables for Meeting_Updates...
Schema retrieved for Meeting_Updates in Meeting_Updates

Fetching tables for road...
Schema retrieved for road in road

Fetching tables for Stock_Ageing...
Schema retrieved for Stock_Ageing in Stock_Ageing

Fetching tables for titanic...
Schema retrieved for titanic in titanic

Fetching tables for tps_trip_details...
Schema retrieved for tps_trip_details in tps_tri

In [7]:
# Step 5: Create LangGraph Query State Schema
class QueryState(BaseModel):
    query: str
    selected_db: str = None
    schema: dict = None
    sql_query: str = None
    validated_sql: str = None
    result: str = None
    chatbot_response: str = None

C:\Users\intern.ai5\AppData\Roaming\Python\Python311\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "schema" in "QueryState" shadows an attribute in parent "BaseModel"
  warnings.warn(


QueryState class is used to define the structure of the state that will be passed through each step of your workflow. Here’s why each part is important:
1. BaseModel: This is likely from Pydantic, a library that helps with data validation and settings management using Python type annotations. It helps in easily serialized to JSON or dictionaries, Data Parsing and reading JSON Schema useful for documentation.
2. query: str: This field stores the natural language query provided by the user.
3. schema: dict: This field holds the database schema information, which is necessary for generating the correct SQL query.
4. sql_query: str = None: This field will store the SQL query generated from the natural language query. It’s initialized to None because it will be populated later in the workflow.
5. validated_sql: str = None: This field will store the validated SQL query after it has been checked for correctness. It’s also initialized to None for the same reason.
6. result: str = None: This field will store the result of the executed SQL query. Again, it’s initialized to None because it will be populated in the final step of the workflow.
7. chatbot_response: str = None: This new field will store the dynamically generated chatbot response based on the query and result. It’s initialized to None because it will be populated after the response is generated.

In [8]:
# Step 6: Define Execution Flow
workflow = StateGraph(QueryState)

StateGraph: A class used to define the execution flow of the workflow, initialized with the QueryState schema.

It is a part of langgraph framework and is used to define, structure and execute the workflows based on state transitions. It act as a Flowchart Engine where:
1. Each node represents a function that transforms data. 
2. Edges define the sequence in which nodes execute.
3. state (data structure) is passed between nodes and updated at each step.

In the following code, it works as:
1. Creates a new state-based graph where Query state is the data strycture that holds information at every step, which means that all nodes in the graph will receive and return an object of type QueryState.

In [9]:
def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union else 0

def select_database(state: QueryState) -> QueryState:
    """Determine the best database and table based on 
    Jaccard similarity with table and column names."""
    query_terms = set(re.findall(r'\b\w+\b', state.query.lower()))
    db_scores = {}
    table_scores = {}

    for db_name, tables in schema_info.items():
        max_table_score = 0
        best_table = None
        for table, schema in tables.items():
            table_terms = set(table.lower().split("_"))
            table_score = jaccard_similarity(query_terms, table_terms)
            column_terms = set(re.findall(r'\b\w+\b', schema.lower()))
            column_score = jaccard_similarity(query_terms, column_terms)
            total_score = max(table_score, column_score)
            
            if total_score > max_table_score:
                max_table_score = total_score
                best_table = table

        db_scores[db_name] = max_table_score
        table_scores[db_name] = best_table

    selected_db = max(db_scores, key=db_scores.get)
    selected_table = table_scores[selected_db]
    print(f"Database selected: {selected_db}")
    print(f"Table selected: {selected_table}")
    return QueryState(query=state.query, selected_db=selected_db, selected_table=selected_table, schema=schema_info[selected_db])


1. This function is a part of LangGraph execution workflow and is responsible for retrieving DB Schema.
2. "state: QueryState": takes a single argument, state whih is an instance of QueryState Class.
3. " -> Query State": type hint (->) indicates that the function returns an instance of QueryState.
4. type hint (->): Indicate what type of value the function returns. Helps in code readability and debugging. It won't hamper the flow of code but for debug, readable, structured.
5. "return ...": It returns an instance of QueryState class.
6. "query = state.query": Keeps the original user query unchanged
7. "selected_db & selected_table": Based on Jaccard Similarity between User Query and Database Schema, Database and Table is being selected.
8. "schema = schema_info[selected_db]": Assigns the schema_info dictionary which contains schema details of the selected database tables to the schema attribute of new QueryState object.

In [10]:
def generate_sql(state: QueryState) -> QueryState:
    """Generate SQL query from the user's query."""
    sql_query = llm.invoke(f"""
        Convert this natural language query into SQL:
        === DATABASE SCHEMA ===
        {state.schema}  
        === USER QUESTION ===
        "{state.query}"
        - Database name and column names can be same.
        - Ensure all relevant columns are considered
        - Do not write column name instead of table name. It generally happens when database table with large number of column names are present.
        - Use appropriate Filtering if the query implies conditions.
        Return only the SQL query.""").content.strip()
    return QueryState(query=state.query, selected_db=state.selected_db, schema=state.schema, sql_query=sql_query)

generate_sql is responsible for converting Natural Language to SQL using a LLM. It takes a QueryState object as input , it generates an SQL Query based on the database schema, and returns a new QueryState object containing the generate SQL.

1. "state:QueryState": Takes Single Argument, state, which is an instanc of QueryState.
2. "-> Query State": With type hint, it indicates the function returns an instance of QueryState.
3. llm.invoke: It calls the LLM to convert NL to SQL. state.schema helps LLM to understand DB Structures, state.query is the question asked by user.
4. content.strip(): It extract and cleans uneccessary white space.
5. query = state.query: Keeps the original user query
6. schema = state.schema: Retains the database schema
7. sql_query: Stores the generated Query.

This ensures that the next step in the Workflow (SQL Validation) gets a properly formatted SQL Query.

In [11]:
def validate_sql(state: QueryState) -> QueryState:
    db = db_connections[state.selected_db]
    valid_columns = set()
    for schema in state.schema.values():
        valid_columns.update(re.findall(r'(\w+)\s\w+', schema))

    query_columns = set(re.findall(r'\bw+\b', state.sql_query.lower()))
    missing_columns = query_columns - valid_columns

    if missing_columns: 
        raise ValueError("Invalid columns in query: {missing_columns}")
    checker_tool = QuerySQLCheckerTool(db=db, llm=llm)
    validated_sql = checker_tool.run(state.sql_query)
    return QueryState(query=state.query, selected_db=state.selected_db, schema=state.schema, sql_query=state.sql_query, validated_sql=validated_sql)


validate_sql is responsible for checking wheether the generated SQL query is valid before executing it. This helps prevent errors by ensuring that SQL Syntax and structure are correct.

1. "state: StateQuery": Takes QueryState object as input.
2. "-> QueryState": Returns a QueryState object as output.
3. "re.findall(r'\bw+\b', VARIABLE)": Finds all sequences of word characters followed by a space and another word in the given VARIABLE string.
4. checker_tool.run(state.sql_query): Calls the validation tool to check if state.sql_query is correct. state.sql_query is the query generate in generate_sql Function.
5. Return a new QueryState object with output parameters.
6. query = state.query: Keeps the original user question.
7. schema = state.schema: Retains the database schema.
8. sql_query = state.sql_query: Stores the generated SQL Query.
9. validated_sql = validated_sql: Stores the validated SQL or error message.

This ensures that only correct SQL queries are processed to the next step execute_sql.

In [12]:
def execute_sql(state: QueryState) -> QueryState:
    """Execute the SQL query and fetch results."""
    query_tool = QuerySQLDatabaseTool(db=db_connections[state.selected_db])
    result = query_tool.run(state.validated_sql)
    return QueryState(query=state.query, selected_db=state.selected_db, schema=state.schema, sql_query=state.sql_query, validated_sql=state.validated_sql, result=result)


execute_sql function is responsible for the validated SQL query on the connected database and Retrieving the results. It ensures that only validated SQL queries are run, prevnting errors and DB crashes.

1. "state: QueryState": Accepts a QueryState object as input.
2. "-> QueryState": Returns a QueryState object as Output.
3. query_tool.run(state.validated_sql): Runs Validated SQL Query on DB. Contains the SQL query that passed validation in validate_sql. Query as Input in this Function and Data Value as output.
4. query = state.query: Keeps the original user question.
5. schema = state.schema: Retains the database schema.
6. sql_query = state.sql_query: Stores the generated SQL Query.
7. validated_sql = state.validated_sql: Keeps the validated SQL query.
8. result=result: Stores the executed query's result.

This ensures that the next step (format_chatbot_response) gets the correct result to generate NL Response.

In [13]:
def format_chatbot_response(state: QueryState) -> QueryState:
    """Generate a chatbot response based on the query and result."""
    chatbot_response = llm.invoke(f"""
        You are a chatbot. Given the following user query and SQL result, generate a natural conversational response.
        === USER QUESTION ===
        {state.query}
        === SQL RESULT ===
        {state.result}
        Provide a user-friendly response.
    """).content.strip()
    return QueryState(query=state.query, selected_db=state.selected_db, schema=state.schema, sql_query=state.sql_query, validated_sql=state.validated_sql, result=state.result, chatbot_response=chatbot_response)


This function generates human-friendly response based on the SQL Query results, where QueryState is given as input object to the format_chatbot_response and "-> QueryState" will return a new QueryStateObject.

1. llm.invoke: Calls the LLM to process the given prompt.The prompt is firmatted text that instructs the LLM on how to generate the chatbot response. Instruct LLM to behave as ChatBot and provide NL {state.query}, result {state.result} and instruct to provide user friendly response in a prompt. 
2. .content: Extracts the actual text response generated by LLM. 
3. .strip(): Removes any unnecessary leading or trailing whitespace.
4. chatbot_response: Stores a well-formatted, NL response based on the SQL Results.
5. "return ...": A new QueryState object is created with query, schema, sql_query, validated_sql, result as before. The new chat_response, now contains a human friendly message. 

This function does not modify the original state object. Instead, it creates a new QueryState ubstabce with the updated chatbot response

In this code QueryState is a Class which means it is here a Data Model that defines attributes, data types, default values. Object of a class is created when you call a function as object(instance) ==> QueryState(BaseModel), here as state: QueryState(...) is written, so state holds the object which holds specific data. It is updated everytime when it is told to. 

1. Class is a blueprint which defines what data it should hold.
2. Object is an actual instance of that blueprint with specific values.

In [14]:
workflow.add_node("select_database", select_database)
workflow.add_node("generate_sql", generate_sql)
workflow.add_node("validate_sql", validate_sql)
workflow.add_node("execute_sql", execute_sql)
workflow.add_node("format_response", format_chatbot_response)

workflow.set_entry_point("select_database")
workflow.add_edge("select_database", "generate_sql")
workflow.add_edge("generate_sql", "validate_sql")
workflow.add_edge("validate_sql", "execute_sql")
workflow.add_edge("execute_sql", "format_response")

executor = workflow.compile()

Nodes are added to the fraph to define execution order. The process flows from Retrieving Schema --> Generating SQL --> Executing SQL --> Formatting the Chatbot Response.

Then, Compile the workflow for execution.
1. add_node("name", function): A method to add a step function to the workflow. "name" is the string identifier for the step. function is the actual identifier for the step.
2. "get_schema", "generate_sql", "validate_sql", "execute_sql": Names of the nodes.
3. get_schema, generate_sql, validate_sql, execute_sql: Corresponding step functions.
4. set_entry_point: First step in the workflow, from where execution begins. 
5. add_edge("A","B"): It connects 2 steps ensuring one executes after the other between two "name". It allows dynamic workflow as per individual's design.

Why name is in  quotes (" ")?
name is the string identifier for the step. Whereas, function is the actual function that performs the step. It is not the actual function but a reference name for tracking the step. It allows to refer to each step by name when setting up the execution order.

Why not just use function names directly?
"name" allows us to easily modify and reference steps without calling function names. Function can have identical names accirss dufferent workflows, but "name" ebsures unique identification within the StateGraph.


In [28]:
import gradio as gr

# Define the function to handle the chatbot interaction
def chatbot_response(history, user_query):
    if user_query.lower() == "exit":
        return history + [("User", user_query), ("Chatbot", "Chatbot: quits")], history, ""
    result = executor.invoke(QueryState(query=user_query))
    response = result['chatbot_response']
    return history + [("User", user_query), ("Chatbot", response)], history, ""

# Custom CSS for styling
custom_css = """
#chatbot {
    background-color: #f5f5f5;
    border-radius: 10px;
    padding: 10px;
}
"""

# Create the Gradio interface
iface = gr.Blocks(css=custom_css)

with iface:
    gr.Markdown("# SQL Chatbot")
    gr.Markdown("Ask me anything about your databases!")
    chatbot = gr.Chatbot()
    state = gr.State([])
    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Type your message here...")
        txt.submit(chatbot_response, [state, txt], [chatbot, state, txt])

# Launch the Gradio interface
iface.launch()

# Ensure the rest of your code is included here, especially the workflow and executor setup


C:\Users\intern.ai5\AppData\Roaming\Python\Python311\site-packages\gradio\components\chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


Database selected: IFDP
Table selected: Coal_IFDP_Daily_Index_Accuracy_Table


In [15]:
"""
What was the average forecast for API4 in March 2024 in IFDP? 97.73
how many male survived in titanic? 109
In which district, Suratgarh Junction is located in Landmarks? HANUMANGARH
What is the total running average speed of all TPS in TPS Trips? 21.35km/h
How many transporter ar ATMSL? 
wagon id of id = 10036288? first row of atmsl 
Give me all the Customer Name of Zone 5
What is the average of QTY in MT
"""

'\nWhat was the average forecast for API4 in March 2024 in IFDP? 97.73\nhow many male survived in titanic? 109\nIn which district, Suratgarh Junction is located in Landmarks? HANUMANGARH\nWhat is the total running average speed of all TPS in TPS Trips? 21.35km/h\nHow many transporter ar ATMSL? \nwagon id of id = 10036288? first row of atmsl \nGive me all the Customer Name of Zone 5\nWhat is the average of QTY in MT\n'

In [ ]:
while True:
    user_query = input("You: ")
    if user_query.lower() == "exit":
        print("Chatbot: quits")
        break
    result = executor.invoke(QueryState(query=user_query))
    print("You:", user_query)
    print("\nChatbot:", result['chatbot_response'])
    print("\n\n")

# Can be deployed on Streamlit, Gradio, Sharepoint etc. Depending upon the situation

AttributeError: 'Textbox' object has no attribute 'style'

In [16]:
user_query = "What was the average forecast for API4 in March 2024"
result = executor.invoke(QueryState(query=user_query))
print("\nChatbot Response:", result)
# Yes

Database selected: IFDP
Table selected: Coal_IFDP_Daily_Index_Accuracy_Table

Chatbot Response: {'query': 'What was the average forecast for API4 in March 2024', 'selected_db': 'IFDP', 'schema': {'Coal_IFDP_Daily_Index_Accuracy_Table': '\nCREATE TABLE "Coal_IFDP_Daily_Index_Accuracy_Table" (\n\tindex_id INTEGER, \n\tindex_name TEXT, \n\tdate TEXT, \n\tactuals REAL, \n\tforecast REAL, \n\tmape REAL, \n\taccuracy REAL, \n\tcreated_at TEXT, \n\tid INTEGER, \n\tPRIMARY KEY (id)\n)\n\n/*\n3 rows from Coal_IFDP_Daily_Index_Accuracy_Table table:\nindex_id\tindex_name\tdate\tactuals\tforecast\tmape\taccuracy\tcreated_at\tid\n1\tAPI4\t2024-03-01 00:00:00.000\t93.38\t92.48\t0.96\t99.04\t2024-03-01 00:00:00.000\t251\n1\tAPI4\t2024-03-04 00:00:00.000\t93.29\t93.87\t0.63\t99.37\t2024-03-04 00:00:00.000\t252\n1\tAPI4\t2024-03-05 00:00:00.000\t94.32\t93.67\t0.69\t99.31\t2024-03-05 00:00:00.000\t253\n*/', 'Coal_IFDP_Dynamic_Pricing_MAP_Actual_Pred': '\nCREATE TABLE "Coal_IFDP_Dynamic_Pricing_MAP_Actua

In [16]:
user_query = "What is the Total Sales of Coastal Engergen in Zone 6 whose Discharge Port was Tuticorin"
result = executor.invoke(QueryState(query=user_query))
print("\nChatbot Response:", result)
# Yes

Database selected: Zone_Excel
Table selected: Budget

Chatbot Response: {'query': 'What is the Total Sales of Coastal Engergen in Zone 6 whose Discharge Port was Tuticorin', 'selected_db': 'Zone_Excel', 'schema': {'Budget': '\nCREATE TABLE "Budget" (\n\t"Business Segment" TEXT, \n\t"Zone" TEXT, \n\t"Customer Name" TEXT, \n\t"Trader" TEXT, \n\t"Industry Segment" TEXT, \n\t"Discharge Port" TEXT, \n\t"Load Port Country" TEXT, \n\t"Grade" TEXT, \n\t"Total Sales" TEXT\n)\n\n/*\n3 rows from Budget table:\nBusiness Segment\tZone\tCustomer Name\tTrader\tIndustry Segment\tDischarge Port\tLoad Port Country\tGrade\tTotal Sales\nSales\tZone 6\tCoastal Energen\tSwamy\tIPP\tTuticorin\tINDO\tINDO-4200 GAR\t2000000\nTPH\tZone 5\tMeenakshi Energy\tKasi\tPower\tKrishnapatnam\tNone\tNone\t600000\nSales\tZone 6\tIL&FS\tSriram\tIPP\tKaraikal\tINDO\tINDO-4200 GAR\t450000\n*/', 'Dispatch_Market_Share': '\nCREATE TABLE "Dispatch_Market_Share" (\n\t"Zone" TEXT, \n\t"Port" TEXT, \n\t"Origin" TEXT, \n\t"Grade" F

In [51]:
user_query = "On 15 Nov 2024, What were the Action when Rajeev Dalmia Visited"
result = executor.invoke(QueryState(query=user_query))
print("\nChatbot Response:", result)
# Yes

Database selected: Meeting_Updates
Table selected: Meeting_Updates

Chatbot Response: {'query': 'On 15 Nov 2024, What were the Action when Rajeev Dalmia Visited', 'selected_db': 'Meeting_Updates', 'schema': {'Meeting_Updates': '\nCREATE TABLE "Meeting_Updates" (\n\t"SI No." TEXT, \n\t"Meeting Date" TEXT, \n\t"Zone" TEXT, \n\t"Trader Name" TEXT, \n\t"Customer Name" TEXT, \n\t"Name of the person visited" TEXT, \n\t"Interaction" TEXT, \n\t"Action" TEXT\n)\n\n/*\n3 rows from Meeting_Updates table:\nSI No.\tMeeting Date\tZone\tTrader Name\tCustomer Name\tName of the person visited\tInteraction\tAction\n1\t14-Nov-24\tZone 8\tSuraj\tMB Sponge / MB Ispat\tArvind Agarwal \tVisited their plant. They consume 44 FC domestic coal at a landed cost of ?8,400. They are setting u\tExpected to be Offering 55NAR Cargo by Jan \x9225.\n2\t14-Nov-24\tZone 8\tSuraj\tCP Sponge\tKumar Chand Chawla\tVisited their plant. They currently consume 44 FC domestic coal at a landed cost of ?8,400 for a 100\tDue to the 

In [52]:
user_query = "What are the remarks Particular for Cement Sector"
result = executor.invoke(QueryState(query=user_query))
print("\nChatbot Response:", result)
# No

Database selected: Zone_Excel
Table selected: Updates_&_Issues

Chatbot Response: {'query': 'What are the remarks Particular for Cement Sector', 'selected_db': 'Zone_Excel', 'schema': {'Budget': '\nCREATE TABLE "Budget" (\n\t"Business Segment" TEXT, \n\t"Zone" TEXT, \n\t"Customer Name" TEXT, \n\t"Trader" TEXT, \n\t"Industry Segment" TEXT, \n\t"Discharge Port" TEXT, \n\t"Load Port Country" TEXT, \n\t"Grade" TEXT, \n\t"Total Sales" TEXT\n)\n\n/*\n3 rows from Budget table:\nBusiness Segment\tZone\tCustomer Name\tTrader\tIndustry Segment\tDischarge Port\tLoad Port Country\tGrade\tTotal Sales\nSales\tZone 6\tCoastal Energen\tSwamy\tIPP\tTuticorin\tINDO\tINDO-4200 GAR\t2000000\nTPH\tZone 5\tMeenakshi Energy\tKasi\tPower\tKrishnapatnam\tNone\tNone\t600000\nSales\tZone 6\tIL&FS\tSriram\tIPP\tKaraikal\tINDO\tINDO-4200 GAR\t450000\n*/', 'Dispatch_Market_Share': '\nCREATE TABLE "Dispatch_Market_Share" (\n\t"Zone" TEXT, \n\t"Port" TEXT, \n\t"Origin" TEXT, \n\t"Grade" FLOAT, \n\t"Updating Date" DAT

In [53]:
user_query = "Give me all the Customer Name of Zone 5"
result = executor.invoke(QueryState(query=user_query))
print("\nChatbot Response:", result)

Database selected: Customer_Portfolio
Table selected: Customer_Portfolio

Chatbot Response: {'query': 'Give me all the Customer Name of Zone 5', 'selected_db': 'Customer_Portfolio', 'schema': {'Customer_Portfolio': '\nCREATE TABLE "Customer_Portfolio" (\n\t"S. No." BIGINT, \n\t"Zone" TEXT, \n\t"Account" TEXT, \n\t"Customer name" TEXT, \n\t"Category" TEXT, \n\t"Industry" TEXT, \n\t"Industry Type" TEXT, \n\t"Buying from " TEXT, \n\t"Plant location" TEXT, \n\t"Cluster location" TEXT, \n\t"State" TEXT, \n\t"1st Port" TEXT, \n\t"2nd Port" TEXT, \n\t"Logistic diff (2nd port - 1st port)" FLOAT, \n\t"Current Sponge Capacity (TPD)" FLOAT, \n\t"Expansion Capacity Current FY (TPD)" FLOAT, \n\t"Kiln Units Breakup " TEXT, \n\t"Cement Capacity (MTPA)" TEXT, \n\t"Power Capacity (MW)" TEXT, \n\t"Others/Ex Stock/FOR" TEXT, \n\t"Type of coal " TEXT, \n\t"Origin" TEXT, \n\t"Grade 1" TEXT, \n\t"Imported coal reqmnt (MTPA)" TEXT, \n\t"Domestic coal reqmnt (MTPA)" FLOAT, \n\t"Domestic mine name ( If applica

In [54]:
user_query = "What is the average aging of all stocks of Zone 1"
result = executor.invoke(QueryState(query=user_query))
print("\nChatbot Response:", result)
# Yes

Database selected: Stock_Ageing
Table selected: Stock_Ageing

Chatbot Response: {'query': 'What is the average aging of all stocks of Zone 1', 'selected_db': 'Stock_Ageing', 'schema': {'Stock_Ageing': '\nCREATE TABLE "Stock_Ageing" (\n\t"Date of updating" TEXT, \n\t"Zone" TEXT, \n\t"Port" TEXT, \n\t"Origin" TEXT, \n\t"Grade" TEXT, \n\t"Remarks" TEXT, \n\t"Vessel Name" TEXT, \n\t"Arrival Date " TEXT, \n\t"Discharge Completion Date" TEXT, \n\t"B/L Qty (MT)" TEXT, \n\t"Dispatch Qty till date (MT)" TEXT, \n\t"Today\'s stock" TEXT, \n\t"Aging" FLOAT, \n\t"Type" TEXT\n)\n\n/*\n3 rows from Stock_Ageing table:\nDate of updating\tZone\tPort\tOrigin\tGrade\tRemarks\tVessel Name\tArrival Date \tDischarge Completion Date\tB/L Qty (MT)\tDispatch Qty till date (MT)\tToday\'s stock\tAging\tType\n6-Dec-24\tZone 1\tTUNA\tUSA\tUS (6900)-401\tNone\tMV MARAN MARINER\t11-May-24\t24-May-24\t1,23,712\t1,08,776\t14,936\t192.0\tSNS\n6-Dec-24\tZone 1\tTUNA\tUSA\tUS (6900)-401\tNone\tMV KM OSAKA\t12-Jul-24\t13-J

In [55]:
user_query = "What is the action for 14 Nov 2024 in Zone 8  with Customer CB Sponge"
result = executor.invoke(QueryState(query=user_query))
print("\nChatbot Response:", result)
# No

Database selected: Meeting_Updates
Table selected: Meeting_Updates

Chatbot Response: {'query': 'What is the action for 14 Nov 2024 in Zone 8  with Customer CB Sponge', 'selected_db': 'Meeting_Updates', 'schema': {'Meeting_Updates': '\nCREATE TABLE "Meeting_Updates" (\n\t"SI No." TEXT, \n\t"Meeting Date" TEXT, \n\t"Zone" TEXT, \n\t"Trader Name" TEXT, \n\t"Customer Name" TEXT, \n\t"Name of the person visited" TEXT, \n\t"Interaction" TEXT, \n\t"Action" TEXT\n)\n\n/*\n3 rows from Meeting_Updates table:\nSI No.\tMeeting Date\tZone\tTrader Name\tCustomer Name\tName of the person visited\tInteraction\tAction\n1\t14-Nov-24\tZone 8\tSuraj\tMB Sponge / MB Ispat\tArvind Agarwal \tVisited their plant. They consume 44 FC domestic coal at a landed cost of ?8,400. They are setting u\tExpected to be Offering 55NAR Cargo by Jan \x9225.\n2\t14-Nov-24\tZone 8\tSuraj\tCP Sponge\tKumar Chand Chawla\tVisited their plant. They currently consume 44 FC domestic coal at a landed cost of ?8,400 for a 100\tDue t

In [ ]:
user_query = "What is the time taken between sjq arrival and sjq departure for vehicle id = 51154"
result = executor.invoke(QueryState(query=user_query))
print("\nChatbot Response:", result)

Database selected: atmspl_trip_details
Table selected: atmspl_trip_details


In [ ]:
user_query = "How many Sales & Enquiry are concluded"
result = executor.invoke(QueryState(query=user_query))
print("\nChatbot Response:", result)
# No

Database selected: Sales_&_Enquiry
Table selected: Sales_&_Enquiry

Chatbot Response: {'query': 'How many Sales & Enquiry are concluded', 'selected_db': 'Sales_&_Enquiry', 'schema': {'Sales_&_Enquiry': '\nCREATE TABLE "Sales_&_Enquiry" (\n\t"S. No" BIGINT, \n\t"FY" TEXT, \n\t"Enquiry Month" TEXT, \n\t"Zone" TEXT, \n\t"Trader Name" TEXT, \n\t"Status (Concluded/Lost/Dropped)" TEXT, \n\t"Name of the Customer" TEXT, \n\t"Trader/End User" TEXT, \n\t"Date" TEXT, \n\t"Enquiry Qty (MT)" FLOAT, \n\t"Grade" TEXT, \n\t"Low/Mid/High" TEXT, \n\t"Origin" TEXT, \n\t"Concluded basis " TEXT, \n\t"Concluded port" TEXT, \n\t"Supplier Name" TEXT, \n\t"Concluded Rate (Rs/$)" TEXT, \n\t"Concluded Cargo" TEXT, \n\t"Discounted sector (Yes/No)" TEXT, \n\t"Discounted sector(Name)" TEXT, \n\t"Adani Offered Rate" TEXT, \n\t"Adani Offered Cargo" TEXT, \n\t"Competitor Concluded Rate" TEXT, \n\t"Deal with Adani/Non Adani" TEXT, \n\t"Concluded With (Competitor)" TEXT, \n\t"Reason" TEXT, \n\t"Detailed Reasons if any" 

In [ ]:
user_query = "How many transporter are ATMSL?"
result = executor.invoke(QueryState(query=user_query))
print("\nChatbot Response:", result)

Database selected: atmspl_trip_details

Chatbot Response: {'query': 'How many transporter are ATMSL?', 'selected_db': 'atmspl_trip_details', 'schema': {'atmspl_trip_details': '\nCREATE TABLE atmspl_trip_details (\n\tsn BIGINT, \n\ttransporter TEXT, \n\tid BIGINT, \n\tvehicle_id BIGINT, \n\ttrip_date TEXT, \n\tvehicle_type TEXT, \n\tvehicle BIGINT, \n\twagon_no FLOAT, \n\tn_box_no TEXT, \n\tengine_no TEXT, \n\trake_no BIGINT, \n\tfnr_no BIGINT, \n\tcoal_type TEXT, \n\tdestination TEXT, \n\tlast_siding_location TEXT, \n\tno_of_extraneous_wagon BIGINT, \n\tdevice_attachment_location TEXT, \n\tsjq_arrival TEXT, \n\tsjq_departure TEXT, \n\tsjq_idle_hrs FLOAT, \n\tavg_speed_sjq_pasla FLOAT, \n\tempty_pasla_in TEXT, \n\tempty_pasla_out TEXT, \n\tempty_pasla_gap FLOAT, \n\tavg_speed_pasla_sonpur FLOAT, \n\tempty_sonpur_in TEXT, \n\tavg_speed_ramanujnagar_sonpur FLOAT, \n\tround_trip_distance FLOAT, \n\tround_trip_duration BIGINT, \n\tcomments TEXT, \n\tis_internal TEXT\n)\n\n/*\n3 rows from at

In [ ]:
'''
In which district, Suratgarh Junction is located in Landmarks? HANUMANGARH
What is the total running average speed of all TPS in TPS Trips? 21.35km/h
How many transporter are ATMSL? 4
'''

'\nIn which district, Suratgarh Junction is located in Landmarks? HANUMANGARH\nWhat is the total running average speed of all TPS in TPS Trips? 21.35km/h\nHow many transporter ar ATMSL? \n'